In [ ]:
# !pip install transformers accelerate
# !pip install datasets
# !pip install wandb

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
import wandb
from datasets import load_metric
import numpy as np

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data loading

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset2.csv')

In [ ]:
df

,id,followers_count,verified,reactions_count,retweet_count,text,source,description,replies_count,fake
0,498254340310966273,1080,False,53,286,Michael Brown is the 17 yr old boy who was sho...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Sr.Lecturer @IEDP_PennGSE | GlobalDev + Migrat...,0,True
1,498260814487642112,1639,False,1,1,@AmeenaGK @jaythenerdkid And how long before a...,"<a href=""http://twitter.com/download/iphone"" r...",#TeamRocketRaccoon #TeamEternia #TeamEtheria #...,0,True
2,498261272174284801,6612,False,0,0,@d_m_elms @AmeenaGK they'll drag his entire hi...,"<a href=""http://twitter.com/download/android"" ...",muslim cis queer neuroatypical intersectional ...,0,True
3,498261302733967361,1070,False,1,2,@d_m_elms @jaythenerdkid or traces back some b...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Sr.Lecturer @IEDP_PennGSE | GlobalDev + Migrat...,0,True
4,498261642099326976,1639,False,5,9,@AmeenaGK @jaythenerdkid CHILD KILLED TO PROTE...,"<a href=""http://twitter.com/download/iphone"" r...",#TeamRocketRaccoon #TeamEternia #TeamEtheria #...,0,True
...,...,...,...,...,...,...,...,...,...,...
105349,581329886599622656,48,False,0,0,@geoffjohns @grantgust Keep quiet for a minute...,"<a href=""http://twitter.com/download/android"" ...",I'm a dreamer...And my heart belongs to @CW_Ar...,0,False
105350,581373405527556096,1283,False,1,0,@geoffjohns 🙏😇,"<a href=""http://twitter.com/download/android"" ...","a student of art, comic book lettering, and gr...",0,False
105351,581387750047383552,1772,False,0,0,@geoffjohns @grantgust :(,"<a href=""http://twitter.com/download/android"" ...",the saints follow me and more awesome people! ...,0,False
105352,582053224645021696,76,False,0,0,@geoffjohns Germanwings A320's crash? http://t...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,0,False


In [ ]:
df['class'] = np.where(df['fake']==True, 1, 0)

W/o metadata

In [ ]:
df["text_and_desc"] = df["text"] + df["description"]

With metadata

In [ ]:
df["verified"] = df["verified"].astype(float)
df["followers_count"] = df["followers_count"].astype(float)
df["reactions_count"] = df["reactions_count"].astype(float)
df["retweet_count"] = df["retweet_count"].astype(float)
df["replies_count"] = df["replies_count"].astype(float)

In [ ]:
df["verified"] = df["verified"].astype(str)
df["followers_count"] = df["followers_count"].astype(str)
df["reactions_count"] = df["reactions_count"].astype(str)
df["retweet_count"] = df["retweet_count"].astype(str)
df["replies_count"] = df["replies_count"].astype(str)

In [ ]:
df["text_and_desc"] = df["text"] + df["description"] + df['followers_count'] + df["verified"] + df["reactions_count"] + df["retweet_count"] + df["replies_count"]
df.head()

,id,followers_count,verified,reactions_count,retweet_count,text,source,description,replies_count,fake,class,text_and_desc
0,498254340310966273,1080.0,0.0,53.0,286.0,Michael Brown is the 17 yr old boy who was sho...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Sr.Lecturer @IEDP_PennGSE | GlobalDev + Migrat...,0.0,True,1,Michael Brown is the 17 yr old boy who was sho...
1,498260814487642112,1639.0,0.0,1.0,1.0,@AmeenaGK @jaythenerdkid And how long before a...,"<a href=""http://twitter.com/download/iphone"" r...",#TeamRocketRaccoon #TeamEternia #TeamEtheria #...,0.0,True,1,@AmeenaGK @jaythenerdkid And how long before a...
2,498261272174284801,6612.0,0.0,0.0,0.0,@d_m_elms @AmeenaGK they'll drag his entire hi...,"<a href=""http://twitter.com/download/android"" ...",muslim cis queer neuroatypical intersectional ...,0.0,True,1,@d_m_elms @AmeenaGK they'll drag his entire hi...
3,498261302733967361,1070.0,0.0,1.0,2.0,@d_m_elms @jaythenerdkid or traces back some b...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Sr.Lecturer @IEDP_PennGSE | GlobalDev + Migrat...,0.0,True,1,@d_m_elms @jaythenerdkid or traces back some b...
4,498261642099326976,1639.0,0.0,5.0,9.0,@AmeenaGK @jaythenerdkid CHILD KILLED TO PROTE...,"<a href=""http://twitter.com/download/iphone"" r...",#TeamRocketRaccoon #TeamEternia #TeamEtheria #...,0.0,True,1,@AmeenaGK @jaythenerdkid CHILD KILLED TO PROTE...


In [ ]:
news_df = df[['text_and_desc', 'class']]

In [ ]:
news_df.fillna('',inplace=True)

<ipython-input-12-4f5679fe4a1b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.fillna('',inplace=True)


In [ ]:
news_df

,text_and_desc,class
0,Michael Brown is the 17 yr old boy who was sho...,1
1,@AmeenaGK @jaythenerdkid And how long before a...,1
2,@d_m_elms @AmeenaGK they'll drag his entire hi...,1
3,@d_m_elms @jaythenerdkid or traces back some b...,1
4,@AmeenaGK @jaythenerdkid CHILD KILLED TO PROTE...,1
...,...,...
105349,@geoffjohns @grantgust Keep quiet for a minute...,0
105350,"@geoffjohns 🙏😇a student of art, comic book let...",0
105351,@geoffjohns @grantgust :(the saints follow me ...,0
105352,,0


#TRAIN

In [ ]:
seed = 42

features = news_df['text_and_desc'].tolist()
targets = news_df['class'].tolist()

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.20, random_state=seed)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=seed)

In [ ]:
X_test

In [ ]:
class NewsDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # convert to pytorch tensors
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# DistilBERT/BERT

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# model_name = "distilbert-base-uncased"
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# tokenizer = AutoTokenizer.from_pretrained(model_name)


# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=300)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=300)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=300)

In [ ]:
train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)
val_dataset = NewsDataset(val_encodings, y_val)

In [ ]:
acc_metric = load_metric("accuracy")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")
f1_metric = load_metric("f1")

# we'll log several evaluation metrics as accuracy alone does not show the whole picture
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = acc_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    return {"acc": acc, "precision": precision, "recall": recall, "f1 score": f1}

<ipython-input-21-dc1db7a3446e>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  acc_metric = load_metric("accuracy")


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=128,
    #warmup_steps=500,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy='epoch',

    #eval_steps=100,
    #logging_steps=100,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: vshishkin. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [ ]:
eval_res = trainer.evaluate(test_dataset)

In [ ]:
eval_res

In [ ]:
model

In [ ]:
import json
with open(f"eval_res.txt", 'w+') as f:
  f.write(json.dumps(eval_res))


# Checkpoint evaluation

In [ ]:
model_name = "/content/results/checkpoint-3372"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=128,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
eval_res = trainer.evaluate(test_dataset)

In [ ]:
eval_res